In [1]:
import ipyleaflet
import ipywidgets
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import datetime

In [2]:
path_inventario = '../LHC_Hidroweb/invent2.csv'

In [3]:
df = pd.read_csv(path_inventario, engine='python', sep='\t', delimiter=';', parse_dates=['UltimaAtualizacao'])

In [4]:
df.columns

Index(['RegistroID', 'Importado', 'Temporario', 'Removido',
       'ImportadoRepetido', 'BaciaCodigo', 'SubBaciaCodigo', 'RioCodigo',
       'EstadoCodigo', 'MunicipioCodigo', 'ResponsavelCodigo',
       'ResponsavelUnidade', 'ResponsavelJurisdicao', 'OperadoraCodigo',
       'OperadoraUnidade', 'OperadoraSubUnidade', 'TipoEstacao', 'Codigo',
       'Nome', 'CodigoAdicional', 'Latitude', 'Longitude', 'Altitude',
       'AreaDrenagem', 'TipoEstacaoEscala', 'TipoEstacaoRegistradorNivel',
       'TipoEstacaoDescLiquida', 'TipoEstacaoSedimentos',
       'TipoEstacaoQualAgua', 'TipoEstacaoPluviometro',
       'TipoEstacaoRegistradorChuva', 'TipoEstacaoTanqueEvapo',
       'TipoEstacaoClimatologica', 'TipoEstacaoPiezometria',
       'TipoEstacaoTelemetrica', 'PeriodoEscalaInicio', 'PeriodoEscalaFim',
       'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
       'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
       'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
      

In [5]:
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].apply(lambda x: x.str.replace(',','.'))
df['Latitude'] = df['Latitude'].astype('float')
df['Longitude'] = df['Longitude'].astype('float')

In [6]:
df[['Latitude','Longitude']].head()

,Latitude,Longitude
0,-13.6500,-73.0833
1,-13.0167,-73.9833
2,-13.6500,-73.3667
3,-13.4333,-73.8333
4,-13.6500,-73.9500


In [7]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude,df.Latitude), crs='epsg:4674')

In [8]:
m01 = ipyleaflet.Map(zoom=2)

# Layers
## All points for HEATMAP
heatmap_all = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],
                                 radius=30, name='All point Heatmap')
m01.add_layer(heatmap_all)

radius_slider = ipywidgets.IntSlider(description='Radius', min=1, max=50, value=15)
ipywidgets.jslink((radius_slider,'value'),(heatmap_all,'radius'))
widget_control01 = ipyleaflet.WidgetControl(widget=radius_slider, position='bottomright')
m01.add_control(widget_control01)

## Last collection day
def date_location(_):
    heatmap_byLast.locations = [tuple(s) for s in df.loc[df['UltimaAtualizacao'] > date_slider.value, ['Latitude','Longitude']].to_numpy()]

heatmap_byLast = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],
                                    radius=30, name='By Date')
m01.add_layer(heatmap_byLast)

ipywidgets.jslink((radius_slider,'value'),(heatmap_byLast,'radius'))


date_slider = ipywidgets.SelectionSlider(options=pd.date_range(start='2000-01-01',end='2020-01-01', freq='M').to_numpy())
date_slider.observe(date_location,names='value')
widget_control02 = ipyleaflet.WidgetControl(widget=date_slider, position='topright')
m01.add_control(widget_control02)


teste = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2020-01-30")


# Controles
## Layer
control_layer = ipyleaflet.LayersControl(position='topright')
m01.add_control(control_layer)

## Split Layer
# split_layer = ipyleaflet.SplitMapControl(left_layer=teste, right_layer=heatmap_byLast)
# m01.add_control(split_layer)

## Fullscreen
control_fullscreen = ipyleaflet.FullScreenControl()
m01.add_control(control_fullscreen)

## Desenho
control_desenho = ipyleaflet.DrawControl()
feature_collection = {'type': 'FeatureCollection',
                      'features': []}
def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""   
    feature_collection['features'].append(geo_json)

control_desenho.on_draw(handle_draw)
m01.add_control(control_desenho)

## Mediçao
# control_medicao = ipyleaflet.MeasureControl(position='bottomleft',
#                                             active_color='orange',
#                                             primary_length_units='kilometers')
# m01.add_control(control_medicao)
# control_medicao.complete_color = 'red'

## Escala
control_escala = ipyleaflet.ScaleControl(position='bottomleft')
m01.add_control(control_escala)

m01

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [38]:
try:
    last_draw = feature_collection['features'][-1]['geometry']
    last_polygon = Polygon([(i[0], i[1]) for i in last_draw['coordinates'][0]])
    print(last_polygon)
except:
    print('Nenhum polígono desenhado')

POLYGON ((-51.833496 -7.710992, -23.005371 -21.943046, -15.270996 2.108899, -46.560059 16.299051, -51.833496 -7.710992))


In [39]:
gdf.loc[(gdf['geometry'].within(last_polygon)),['Nome', 'Codigo']]

,Nome,Codigo
82,SÍTIO BICA,438038
84,BIXOPA,438074
85,TRUSSU,639050
86,LIMOEIRO DO NORTE,538009
87,FAZENDA MORADA NOVA,639075
...,...,...
36050,SÃO JOÃO DO PIAUÍ,842011
36084,CORURIPE,1036012
36085,EPITÁCIO PESSOA (COLÔNIA),1037067
36086,MATA DE SÃO JOÃO,1238030


In [40]:
gdf.loc[(gdf['geometry'].within(last_polygon)) & (gdf['UltimaAtualizacao']>date_slider.value) ,['Nome', 'Codigo']]

,Nome,Codigo
82,SÍTIO BICA,438038
84,BIXOPA,438074
85,TRUSSU,639050
86,LIMOEIRO DO NORTE,538009
87,FAZENDA MORADA NOVA,639075
...,...,...
36050,SÃO JOÃO DO PIAUÍ,842011
36084,CORURIPE,1036012
36085,EPITÁCIO PESSOA (COLÔNIA),1037067
36086,MATA DE SÃO JOÃO,1238030


In [24]:
path_shapefile = r'G:\Meu Drive\USP-SHS\Documentos_Word_Excel_PowerPoint\Shapefiles\35MUE250GC_SIR.shp'
# path_shapefile = r'G:\Meu Drive\USP-SHS\Documentos_Word_Excel_PowerPoint\Shapefiles\bioma.shp'

In [29]:
teste_shp = gpd.read_file(path_shapefile)
city = teste_shp.loc[teste_shp['NM_MUNICIP']=='DRACENA']
city

,NM_MUNICIP,CD_GEOCMU,geometry
2,DRACENA,3514403,"POLYGON ((-51.48515 -21.42765, -51.48500 -21.4..."


In [44]:
for i in city['geometry']:
    print(gdf.loc[gdf['geometry'].within(i), ['Nome', 'Codigo']])

                        Nome   Codigo
4912   DRACENA_Jardim Europa  2151062
12948                DRACENA  2151010
